In [243]:
from binance.client import Client
import pandas as pd
import numpy as np
import math
import time
import statistics
#import coin_data
from config import api_key, api_secret

In [244]:
data = {
    "ROSEUSDT": {
        "i1": 0.001,
        "i2": 0.0001,
    },
    "MANAUSDT": {
        "i1": 0.01, 
        "i2": 0.001,
        "sl": 0.3,
    },
    "ADAUSDT": {
        "i1": 0.01, 
        "i2": 0.001,
    },
    "BTCUSDT": {
        "i1": 100,
        "i2": 50,
    },
    "MATICUSDT": {
        "i1": 0.01,
        "i2": 0.001,
    },
}

In [245]:
client = Client(api_key=api_key, api_secret= api_secret)
ticker = 'MANAUSDT'

# 1. Extraer datos de la moneda

- Traemos toda la info de todas las monedas de futuros de binance

In [246]:
info = client.futures_exchange_info()

- Buscamos la moneda que necesitamos
- Y extraemos la información que creamos necesaria
- Esto lo deberíamos ejecutar 1 sola vez para todas las monedas y guardarlo para solo consultarlo como diccionario y no extraer la información cada vez
- Aunque los datos traen la precisión para el precio, en algunas monedas está mal. Por lo tanto, se crea la función que retorna la cantidad de decimales a través de otra propiedad listada

In [247]:
def get_precision(texto:str) -> int:
    for i in range(2, len(texto)):
        if texto[i] == '1':
            return i-1

In [248]:
for d in info['symbols']:
    if d['pair']==ticker:
        print(d)
        print("")
        print(f"Moneda: {d['pair']}\nSubtipo: {d['underlyingSubType']}\nPrecisión Precio: {get_precision(d['filters'][0]['tickSize'])} decimales")


{'symbol': 'MANAUSDT', 'pair': 'MANAUSDT', 'contractType': 'PERPETUAL', 'deliveryDate': 4133404800000, 'onboardDate': 1615705200000, 'status': 'TRADING', 'maintMarginPercent': '2.5000', 'requiredMarginPercent': '5.0000', 'baseAsset': 'MANA', 'quoteAsset': 'USDT', 'marginAsset': 'USDT', 'pricePrecision': 4, 'quantityPrecision': 0, 'baseAssetPrecision': 8, 'quotePrecision': 8, 'underlyingType': 'COIN', 'underlyingSubType': ['Metaverse'], 'settlePlan': 0, 'triggerProtect': '0.0500', 'liquidationFee': '0.010000', 'marketTakeBound': '0.10', 'filters': [{'minPrice': '0.0136', 'maxPrice': '100000', 'filterType': 'PRICE_FILTER', 'tickSize': '0.0001'}, {'stepSize': '1', 'filterType': 'LOT_SIZE', 'maxQty': '1000000', 'minQty': '1'}, {'stepSize': '1', 'filterType': 'MARKET_LOT_SIZE', 'maxQty': '100000', 'minQty': '1'}, {'limit': 200, 'filterType': 'MAX_NUM_ORDERS'}, {'limit': 10, 'filterType': 'MAX_NUM_ALGO_ORDERS'}, {'notional': '5', 'filterType': 'MIN_NOTIONAL'}, {'multiplierDown': '0.9000', 'm

# 2. Obtener Shock Points

# a. Libro de ordenes

- Vamos a obtener el libro de órdenes de la moneda
- Luego separamos las ventas de las compras y las colocamos en un dataframe para mejor organización
- Como nota, lo que trae el libro no es la cantidad total de monedas por bloque sino el total de USDT
- En el caso de las ventas, para mostrarlo como lo vemos en Binance (mayor a menor) ordenamis descebdentemente

In [249]:
order_book = client.futures_order_book(symbol=ticker, limit=1000)
#order_book

In [250]:
ventas = order_book['asks']
ventas = pd.DataFrame(ventas, columns=['precio','total_usdt']).sort_index(0, ascending=False)
ventas

/tmp/ipykernel_26903/4237633481.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  ventas = pd.DataFrame(ventas, columns=['precio','total_usdt']).sort_index(0, ascending=False)


,precio,total_usdt
999,1.1504,158
998,1.1503,78
997,1.1500,2312
996,1.1498,7
995,1.1497,44
...,...,...
4,1.0130,16482
3,1.0129,10627
2,1.0128,8988
1,1.0127,5856


In [251]:
compras = order_book['bids']
compras = pd.DataFrame(compras, columns=['precio','total_usdt'])
compras

,precio,total_usdt
0,1.0125,1384
1,1.0124,3668
2,1.0123,5029
3,1.0122,5061
4,1.0121,4092
...,...,...
995,0.9002,1308
996,0.9001,333
997,0.9000,39315
998,0.8999,1277


## b. Obtención de rangos de precios

- Para obtener los rangos y obtener los puntos a cómo lo hace el gafas, debemos crear rangos
- Para crear los rangos necesitamos el precio mínimo y máximo para redondearlos hacia abajo y arriba respecto a los múltiplos de los libros

### i. Obtención de máximos y mínimos precios

- Para poder hacer esto, tenemos que decirle al Dataframe qué tipo de valores son las columnas
- Esto es porque, como se puede observar a continuación, los marca como object

In [252]:
compras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   precio      1000 non-null   object
 1   total_usdt  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


- Para modificar el tipo de objetos de las columnas, hacemos:

In [253]:
ventas[['precio','total_usdt']] = ventas[['precio','total_usdt']].astype('float')
ventas

,precio,total_usdt
999,1.1504,158.0
998,1.1503,78.0
997,1.1500,2312.0
996,1.1498,7.0
995,1.1497,44.0
...,...,...
4,1.0130,16482.0
3,1.0129,10627.0
2,1.0128,8988.0
1,1.0127,5856.0


In [254]:
compras[['precio','total_usdt']] = compras[['precio','total_usdt']].astype('float')
compras

,precio,total_usdt
0,1.0125,1384.0
1,1.0124,3668.0
2,1.0123,5029.0
3,1.0122,5061.0
4,1.0121,4092.0
...,...,...
995,0.9002,1308.0
996,0.9001,333.0
997,0.9000,39315.0
998,0.8999,1277.0


- Ahora sí, obtenemos los máximos y mínimos

In [255]:
max_ventas = ventas['precio'].max()
min_ventas = ventas['precio'].min()
print('Ventas(mín-máx): {} - {}'.format(min_ventas, max_ventas))

max_compras = compras['precio'].max()
min_compras = compras['precio'].min()
print('Compras(mín-máx): {} - {}'.format(min_compras, max_compras))

Ventas(mín-máx): 1.0126 - 1.1504
Compras(mín-máx): 0.8998 - 1.0125


### ii. Obtención de Dataframes con los Rangos

- Para esto, necesitamos tener los múltiplos de las moendas cómo lo hace el gafas
- Desafortunadamante, aún no encuentro cómo obtener los múltiplos de binance por lo que toca escribirlos en el archivo coin_data.py
- Para obtener el inicio de inicio y final de un rango, hacemos:

1. (precio_minimo / multiplo) ; donde el múltiplo es el que obtenemos de coind_data en i1
2. Luego redondeamos al entero inferior con math.floor()
3. Y multiplicamos por el múltiplo nuevamente para obtener el inicio del rango
4. (precio_maximo / multiplo)
5. redondeamos hacia el entero superior con math.ceil()
6. Y multiplicamos por el múltiplo nuevamente para obtener el máximo del rango

#### Obtención de valores iniciales y finales de rango con primer múltiplo

In [256]:
# Ventas
inicio_rango_ventas_1 = ( math.floor( (min_ventas / data[ticker]['i1']) ) ) * data[ticker]['i1']
final_rango_ventas_1 = ( math.ceil( (max_ventas / data[ticker]['i1']) ) ) * data[ticker]['i1']
print('Rango Ventas: {} - {}'.format(inicio_rango_ventas_1,final_rango_ventas_1))
# Compras
inicio_rango_compras_1 = ( math.floor( (min_compras / data[ticker]['i1']) ) ) * data[ticker]['i1']
final_rango_compras_1 = ( math.ceil( (max_compras / data[ticker]['i1']) ) ) * data[ticker]['i1']
print('Rango Compras: {} - {}'.format(inicio_rango_compras_1,final_rango_compras_1))

Rango Ventas: 1.01 - 1.16
Rango Compras: 0.89 - 1.02


#### Obtención de valores iniciales y finales de rango con segundo múltiplo

In [257]:
# Ventas
inicio_rango_ventas_2 = ( math.floor( (min_ventas / data[ticker]['i2']) ) ) * data[ticker]['i2']
final_rango_ventas_2 = ( math.ceil( (max_ventas / data[ticker]['i2']) ) ) * data[ticker]['i2']
print('Rango Ventas: {} - {}'.format(inicio_rango_ventas_2,final_rango_ventas_2))
# Compras
inicio_rango_compras_2 = ( math.floor( (min_compras / data[ticker]['i2']) ) ) * data[ticker]['i2']
final_rango_compras_2 = ( math.ceil( (max_compras / data[ticker]['i2']) ) ) * data[ticker]['i2']
print('Rango Compras: {} - {}'.format(inicio_rango_compras_2,final_rango_compras_2))

Rango Ventas: 1.012 - 1.151
Rango Compras: 0.899 - 1.0130000000000001


### iii. Obtención de los dataframes agrupados por rangos

- Ya que tenemos los valores inicial y final de los rangos, creamos el Dataframe con datos agrupados por rangos
- Solo que, antes tenemos que tener un rango con incrementos de acuerdo al múltiplo. Para esto usamos np.arange()

- Con múltiplo 1

In [258]:
rango_v_1 = np.arange(inicio_rango_ventas_1, final_rango_ventas_1+data[ticker]['i1'], data[ticker]['i1'])
ventas_ag_1 = ventas.groupby(pd.cut(ventas.precio,rango_v_1)).sum()
print('Ventas agrupadas segun incremento 1')
print(rango_v_1)
print(ventas_ag_1.sort_index(ascending=False))
rango_c_1 = np.arange(inicio_rango_compras_1, final_rango_compras_1+data[ticker]['i1'], data[ticker]['i1'])
compras_ag_1 = compras.groupby(pd.cut(compras.precio,rango_c_1)).sum()
print('\nCompras agrupadas segun incremento 1')
print(rango_c_1)
print(compras_ag_1.sort_index())

Ventas agrupadas segun incremento 1
[1.01 1.02 1.03 1.04 1.05 1.06 1.07 1.08 1.09 1.1  1.11 1.12 1.13 1.14
 1.15 1.16]
               precio  total_usdt
precio                           
(1.15, 1.16]   2.3007       236.0
(1.14, 1.15]  61.8224     33236.0
(1.13, 1.14]  57.8926     28366.0
(1.12, 1.13]  65.2493    112344.0
(1.11, 1.12]  65.7738    239361.0
(1.1, 1.11]   71.8182    251064.0
(1.09, 1.1]   73.3691     97350.0
(1.08, 1.09]  82.4708     58895.0
(1.07, 1.08]  77.4173     80490.0
(1.06, 1.07]  72.4319    127853.0
(1.05, 1.06]  88.6139    253443.0
(1.04, 1.05]  89.8545    292668.0
(1.03, 1.04]  91.0739    334579.0
(1.02, 1.03]  97.3745    807013.0
(1.01, 1.02]  76.2225    687454.0

Compras agrupadas segun incremento 1
[0.89 0.9  0.91 0.92 0.93 0.94 0.95 0.96 0.97 0.98 0.99 1.   1.01 1.02
 1.03]
               precio  total_usdt
precio                           
(0.89, 0.9]    2.6997     40740.0
(0.9, 0.91]   78.7452    122126.0
(0.91, 0.92]  75.0226     76806.0
(0.92, 0.93]  77.

- con múltiplo 2

In [259]:
rango_v_2 = np.arange(inicio_rango_ventas_2, final_rango_ventas_2+data[ticker]['i2'], data[ticker]['i2'])
ventas_ag_2 = ventas.groupby(pd.cut(ventas.precio,rango_v_2)).sum()
print('Ventas agrupadas segun incremento 2')
#print(rango_v_2)
print(ventas_ag_2.sort_index(ascending=False))
rango_c_2 = np.arange(inicio_rango_compras_2, final_rango_compras_2+data[ticker]['i2'], data[ticker]['i2'])
compras_ag_2 = compras.groupby(pd.cut(compras.precio,rango_c_2)).sum()
print('\nCompras agrupadas segun incremento 2')
#print(rango_c_2)
print(compras_ag_2.sort_index(ascending=False))

Ventas agrupadas segun incremento 2
                 precio  total_usdt
precio                             
(1.15, 1.151]    3.4507      2548.0
(1.149, 1.15]    3.4487        69.0
(1.148, 1.149]   5.7421       536.0
(1.147, 1.148]   8.0321      1211.0
(1.146, 1.147]   5.7317       205.0
...                 ...         ...
(1.016, 1.017]  10.1645     38090.0
(1.015, 1.016]  10.1545     91453.0
(1.014, 1.015]  10.1445     82878.0
(1.013, 1.014]   9.1215    115825.0
(1.012, 1.013]   5.0640     41959.0

[139 rows x 2 columns]

Compras agrupadas segun incremento 2
                 precio  total_usdt
precio                             
(1.012, 1.013]   5.0615     19234.0
(1.011, 1.012]  10.1155    151009.0
(1.01, 1.011]   10.1055     89677.0
(1.009, 1.01]   10.0955    112540.0
(1.008, 1.009]  10.0855    149933.0
...                 ...         ...
(0.903, 0.904]   9.0355     11250.0
(0.902, 0.903]   4.5125      1348.0
(0.901, 0.902]   9.0155      4460.0
(0.9, 0.901]     8.1052      2216.0
(0

#### Obtención de los shock points

- Ahora, para obtener los shock point, necesitamos obtener el dato donde el total usdt es máximo
- Traer el índice y extraer:
    - Para ventas: el lado derecho del intervalo. Porque es la parte superior del intervalo
    - Para compras: el lado izquiero del intervalo. Porque es la parte inferior del intervalo

In [260]:
from numpy import sort


sp_v_1 = ventas_ag_1['total_usdt'].idxmax().right
#print(sp_v_1)
#print(ventas_ag_2)
#print(ventas_ag_2['total_usdt'].max())
#print(ventas_ag_2['total_usdt'].idxmax())
#print(ventas_ag_2['total_usdt'].idxmax())
sp_v_2 = ventas_ag_2['total_usdt'].idxmax().right
sp_v = []
sp_v.append(sp_v_1)
sp_v.append(sp_v_2)
sp_v = sort(sp_v)
sp_v_1 = sp_v[1]
sp_v_2 = sp_v[0]
print(f'Ventas:\n***************\n2.  {sp_v_1}\n1.  {sp_v_2}')

Ventas:
***************
2.  1.111
1.  1.03


In [261]:
sp_c_1 = compras_ag_1['total_usdt'].idxmax().left
#print(sp_c_1)
#print(compras_ag_2)
#print(compras_ag_2['total_usdt'].max())
#print(compras_ag_2['total_usdt'].idxmax())
sp_c_2 = compras_ag_2['total_usdt'].idxmax().left
#print(sp_c_2)
sp_c = []
sp_c.append(sp_c_1)
sp_c.append(sp_c_2)
sp_c = sort(sp_c)
sp_c_1 = sp_c[1]
sp_c_2 = sp_c[0]
print(f'Compras:\n***************\n1.  {sp_c_1}\n2.  {sp_c_2}')

Compras:
***************
1.  1.002
2.  1.0


# 3. Analizar Shock Points

- El análisis de los puntos se hará de la siguiente manera:
    - Para ventas, obtenemos la distancia o porcentaje entre puntos de venta: pct_v = ( (venta_fin - venta_inicio)/venta_din ) * 100
    - Para compras, lo calculasmo así: pct_c = ( (compra_inicio - compra_fin)/compra_inicio ) * 100
- Luego, debemos obtener la distancia entre el primer punto de venta y de compras, en este caso tenemos una ligera variación por lo que obtendremos el promedio. 
    - (1)=  (compras_inicio - ventas_inicio)/compras_inicio ) * -100
    - (2)=  (ventas_inicio - compras_inicio)/ventas_inicio ) * 100
    - pct_sep = promedio(1,2)

In [262]:
pct_ventas = round( ( (sp_v_1 - sp_v_2) / sp_v_2 ) * 100, 2 )
print(f'Distancia entre ventas: {pct_ventas}%')

pct_compras = round( ( (sp_c_1 - sp_c_2) / sp_c_1 ) * 100, 2)
print(f'Distancia entre compraas: {pct_compras}%')

pct_sep_1 = (( (sp_c_1 - sp_v_2) / sp_c_1 ) * -100)
pct_sep_2 = ((sp_v_2 - sp_c_1) / sp_v_2 ) * 100
pct_sep = round((pct_sep_1+pct_sep_2)/2,2)
print(f'Porcentaje entre compra y venta: {pct_sep}%')

Distancia entre ventas: 7.86%
Distancia entre compraas: 0.2%
Porcentaje entre compra y venta: 2.76%


# 4. Resultado del Análisis de los Shock Points

- Una vez que tenemos las distancias, procedemos a comparar lo siguiente:
    - Para compras:
        - Distancia entre compra y venta debe ser mayor a la distancia entre compras por 2.5 o más veces
    - Para ventas:
        - - Distancia entre compra y venta debe ser mayor a la distancia entre ventas por 2.5 o más veces
- Si se cumplen esas condiciones enviamos señal para compra y/o venta
- Una nota aquí es que, a la distancia entre ventas o entre compras, habría que aumentarle la distancia entre el segundo shock_point y el SL

In [263]:
flag_compra = False
flag_venta = False
if (pct_sep/pct_ventas) >= 2.5: # Aquí es pct_sep/(pct_ventas+SL)
    print(f'Se puede vender en {ticker}')
    flag_venta = True
if (pct_sep/pct_compras) >= 2.5: # Aquí es pct_sep/(pct_compras+SL)
    print(f'Se puede comprar en {ticker}')
    flag_compra = True

Se puede comprar en MANAUSDT


# 5. Calcular SL y TP

- En el caso que sí se habilite alguna operación debemos tener una gestión por moneda dentro del archivo coin_data para determinar qué tanta separación entre el segundo shock point y nuestro SL debe haber
- Esto lo veremos de acuerdo a nuestro análisis de nuestras monedas teniendo uno por default o uno por moneda

In [266]:
if flag_venta:
    print(f'Entrada Venta:\t${sp_v_2}')

if flag_compra:
    print(f'Entrada Compra:\t${sp_c_1}')

Entrada Compra:	$1.002


# 6. Ejecutar órdenes